In [1]:
import pandas as pd
import numpy as np

In [2]:
def vv4(row):
    row = str(row)
    X = [1,1,1]
    X[0] = row[0:4]
    X[1] = row[4:6]
    X[2] = row[6:8]
    return X[0]+'-'+X[1]+'-'+X[2]

In [3]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"
n = path + '\Cleaned_Stocks_Holders_1399-07-21_From96.csv'
df = pd.read_csv(n)

In [4]:
df = df.drop(df.loc[df['Holder']=='شخص حقیقی'].index)
df = df.drop(df[(df['Trade']=='No')&((df['close_price']==10)|(df['close_price']==1000)|(df['close_price']==10000)|(df['close_price']==100000))].index)
df = df.drop(df[(df['symbol'] == 'وقوام')&(df['close_price'] == 1000)].index)
symbols = [ 'سپرده','هما','وهنر-پذیره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['symbol'].isin(symbols)].index)
df = df.drop(df[(df.symbol == 'اتکای')&(df.close_price == 1000)].index)

In [5]:
df['date1'] = df['date'].apply(vv4)
df['date1'] = pd.to_datetime(df['date1']) 
df['week_of_year'] = df['date1'].dt.week 
df['year_of_year'] = df['date1'].dt.year

In [6]:
df= df[df.Trade == 'Yes']

In [7]:
Holders = list(set(df.Holder))
ids = list(range(len(Holders)))
mapingdict = dict(zip(Holders, ids))
df['Holder_id'] = df['Holder'].map(mapingdict)

symbols = list(set(df.symbol))
ids = list(range(len(symbols)))
mapingdict = dict(zip(symbols, ids))
df['id'] = df['symbol'].map(mapingdict)

In [8]:
pdata = df.groupby(['symbol','date']).first().reset_index()[['symbol', 'date', 'jalaliDate', 'close_price','week_of_year','year_of_year','Trade']]
mapingdict = dict(zip(symbols,ids))
pdata['id'] = pdata['symbol'].map(mapingdict)

In [9]:
MarketCap = df
MarketCap['MarketCap'] = MarketCap['close_price'] * MarketCap['shrout']
MarketCap = MarketCap.groupby(['date','symbol']).first().reset_index()[['date', 'jalaliDate', 'symbol','MarketCap']].groupby(['date','jalaliDate'])['MarketCap'].sum().to_frame().reset_index()
TMarketCap = list(set(MarketCap.MarketCap))
days = list(set(MarketCap.date))
mapingdict = dict(zip(days, TMarketCap))
df['TMarketCap'] = df['date'].map(mapingdict)
df['Size'] = df['MarketCap']/df['TMarketCap']

In [12]:
def FCAPf(S_g,g):
    intersection = list(set.intersection(set(S_g.date), set(g.date)))
    g = g.loc[g.date.isin(intersection)]
    S_g = S_g.loc[S_g.date.isin(intersection)]
    a = g.merge(S_g , on = ['Holder_id','date'])
    if len(a) == 0: 
        return
    else:
        a['FCAPf'] = (a['nshares_x']*a['close_price_x'] + a['nshares_y']*a['close_price_y'])/(a['shrout_x']*a['close_price_x'] + a['shrout_y']*a['close_price_y'])
        a['SizeRatio'] = (a['Size_x'])/(a['Size_y'])
        f = a.groupby(['date','jalaliDate_x','id_x','id_y','week_of_year_x','year_of_year_x','SizeRatio','Size_x','Size_y'])['FCAPf'].sum().to_frame().reset_index().groupby(['year_of_year_x','week_of_year_x']).last().reset_index()
        return f


In [16]:
def FisrStep(g , df):
    F_id = g.id.iloc[0]
    print(F_id)
    Next_df = df[df.id > F_id]
    S_gg = Next_df.groupby(['id'])
    f = S_gg.apply(FCAPf , g = g )
    return f

In [26]:
gg = df.groupby(['id'])
ff = gg.apply(FisrStep , df = df )

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


AttributeError: 'DataFrame' object has no attribute 'id'

In [11]:
F_id = g.id.iloc[0]
Next_df = df[df.id > F_id]
S_gg = Next_df.groupby(['id'])
S_g = S_gg.get_group(1)

In [13]:
S_gg.apply(FCAPf , g = g )

year_of_year_x  week_of_year_x      date  jalaliDate_x  id_x  id_y  \
id                                                                            
6   0              2018               1  20181231      13971010     0     6   
    1              2018               2  20180113      13961023     0     6   
    2              2018               3  20180121      13961101     0     6   
    3              2018               5  20180204      13961115     0     6   
    4              2018               6  20180205      13961116     0     6   
    5              2018               7  20180214      13961125     0     6   
    6              2018              11  20180313      13961222     0     6   
    7              2018              14  20180408      13970119     0     6   
    8              2018              15  20180411      13970122     0     6   
    9              2018              16  20180418      13970129     0     6   
    10             2018              17  20180428      13970208     0     6   
    11             2018              18  20180505      13970215     0     6   
    12             2018              19  20180512      13970222     0     6   
    13             2018              20  20180519      13970229     0     6   
    14             2018              21  20180526      13970305     0     6   
    15             2018              22  20180602      13970312     0     6   
    16             2018              23  20180609      13970319     0     6   
    17             2018              24  20180613      13970323     0     6   
    18             2018              25  20180620      13970330     0     6   
    19             2018              26  20180630      13970409     0     6   
    20             2018              27  20180704      13970413     0     6   
    21             2018              28  20180714      13970423     0     6   
    22             2018              29  20180716      13970425     0     6   
    23             2018              30  20180725      13970503     0     6   
    24             2018              31  20180801      13970510     0     6   
    25             2018              32  20180808      13970517     0     6   
    26             2018              33  20180818      13970527     0     6   
    27             2018              34  20180825      13970603     0     6   
    28             2018              35  20180902      13970611     0     6   
    29             2018              36  20180909      13970618     0     6   
...                 ...             ...       ...           ...   ...   ...   
628 75             2018              41  20181014      13970722     0   628   
    76             2018              42  20181021      13970729     0   628   
    77             2018              45  20181111      13970820     0   628   
    78             2018              46  20181118      13970827     0   628   
    79             2018              47  20181120      13970829     0   628   
    80             2018              48  20181202      13970911     0   628   
    81             2018              49  20181209      13970918     0   628   
    82             2018              50  20181216      13970925     0   628   
    83             2018              51  20181223      13971002     0   628   
    84             2018              52  20181230      13971009     0   628   
    85             2019               1  20190105      13971015     0   628   
    86             2019               2  20190113      13971023     0   628   
    87             2019               3  20190120      13971030     0   628   
    88             2019               4  20190127      13971107     0   628   
    89             2019               5  20190203      13971114     0   628   
    90             2019               6  20190210      13971121     0   628   
    91             2019               7  20190217      13971128     0   628   
    92             2019               8  20190219      1397

In [ ]:
gg = df.groupby(['id'])
ids = list(gg.groups.keys())
data = pd.DataFrame()
for i in ids:
    dfi = df.loc[(df.id == i)]
    pdfi = pdata.loc[(pdata.id == i)]
    print('\n'+str(i))
    for j in ids[i+1:]:
        print(j , end="\r", flush=True)
        if i==j : continue    
        dfj = df.loc[(df.id == j)]
        intersection = list(set.intersection(set(dfi.date), set(dfj.date)))
        dfj = dfj.loc[dfj.date.isin(intersection)]
        dfi = dfi.loc[dfi.date.isin(intersection)]
        a = dfi.merge(dfj , on = ['Holder_id','date'])
        if len(a) == 0: 
#             print('No')
            continue
        a['FCAPf'] = (a['nshares_x']*a['close_price_x'] + a['nshares_y']*a['close_price_y'])/(a['shrout_x']*a['close_price_x'] + a['shrout_y']*a['close_price_y'])
        a['SizeRatio'] = (a['Size_x'])/(a['Size_y'])
        f = a.groupby(['date','jalaliDate_x','id_x','id_y','week_of_year_x','year_of_year_x','SizeRatio','Size_x','Size_y'])['FCAPf'].sum().to_frame().reset_index().groupby(['year_of_year_x','week_of_year_x']).last().reset_index()
        
        
        
        dfj = pdata.loc[(pdata.id == j)]
        intersection = list(set.intersection(set(pdfi.date), set(dfj.date)))
        dfj = dfj.loc[dfj.date.isin(intersection)]
        pdfi = pdfi.loc[pdfi.date.isin(intersection)]
        b = pdfi.merge(dfj , on = ['date'])
        c = b.groupby(['week_of_year_x','year_of_year_x']).last().reset_index()[['week_of_year_x','year_of_year_x','date','jalaliDate_x']]
        
        b['Return_x'] = b['close_price_x'].pct_change(periods = 1)
        b['Return_y'] = b['close_price_y'].pct_change(periods = 1)
        
        ff = b.groupby(['id_x','id_y','week_of_year_x','year_of_year_x'])[['Return_x','Return_y']].corr().reset_index().iloc[:,0:-1]
        ff = ff[ff['level_4'] != 'Return_x'].drop(columns = ['level_4']).rename(columns = {'Return_x':'ρ'}).sort_values(by = ['year_of_year_x','week_of_year_x'])
        ff = ff.merge(c,on = ['year_of_year_x','week_of_year_x'])
        
        d = f.merge(ff,on=['year_of_year_x','week_of_year_x','id_x','id_y','jalaliDate_x','date'],how = 'outer')
        d['FCAPf'] = (d['FCAPf'] - d['FCAPf'].mean()).divide(d['FCAPf'].std())
        d = d.rename(columns = {'FCAPf':'FCAPF'})
        d['SGroup'] = 0
        if a[['group_name_x','group_name_y']].iloc[0][0] == a[['group_name_x','group_name_y']].iloc[0][1]:
            d['SGroup'] = 1
        d['PCorr_1'] = d['ρ'].shift(1)
        d['PCorr_3'] = d['ρ'].shift(3)
        d['PCorr_6'] = d['ρ'].shift(6)
        d['PCorr_12'] = d['ρ'].shift(12)
        d['FCorr_1'] = d['ρ'].shift(-1)
        d['FCorr_3'] = d['ρ'].shift(-3)
        d['FCorr_6'] = d['ρ'].shift(-6)
        d['FCorr_12'] = d['ρ'].shift(-12)
        d = d.loc[~d['FCAPF'].isnull()]
        
        data = data.append(d)

        
mapingdict = dict(zip(ids,symbols))
data['symbol1'] = data['id_x'].map(mapingdict)
data['symbol2'] = data['id_y'].map(mapingdict)

In [ ]:
i,j = 292,540

In [ ]:
        i,j = 489,503
        dfi = df.loc[(df.id == i)]
        pdfi = pdata.loc[(pdata.id == i)]    
        dfj = df.loc[(df.id == j)]
        intersection = list(set.intersection(set(dfi.date), set(dfj.date)))
        dfj = dfj.loc[dfj.date.isin(intersection)]
        dfi = dfi.loc[dfi.date.isin(intersection)]
        a = dfi.merge(dfj , on = ['Holder_id','date'])
#         if len(a) == 0: 
# #             print('No')
#             continue
        a['FCAPf'] = (a['nshares_x']*a['close_price_x'] + a['nshares_y']*a['close_price_y'])/(a['shrout_x']*a['close_price_x'] + a['shrout_y']*a['close_price_y'])
        a['SizeRatio'] = (a['Size_x'])/(a['Size_y'])
        f = a.groupby(['date','jalaliDate_x','id_x','id_y','week_of_year_x','year_of_year_x','SizeRatio','Size_x','Size_y'])['FCAPf'].sum().to_frame().reset_index().groupby(['year_of_year_x','week_of_year_x']).last().reset_index()
        
        
        
        dfj = pdata.loc[(pdata.id == j)]
        intersection = list(set.intersection(set(pdfi.date), set(dfj.date)))
        dfj = dfj.loc[dfj.date.isin(intersection)]
        pdfi = pdfi.loc[pdfi.date.isin(intersection)]
        b = pdfi.merge(dfj , on = ['date'])
        c = b.groupby(['week_of_year_x','year_of_year_x']).last().reset_index()[['week_of_year_x','year_of_year_x','date','jalaliDate_x']]
        
        b['Return_x'] = b['close_price_x'].pct_change(periods = 1)
        b['Return_y'] = b['close_price_y'].pct_change(periods = 1)
        
        ff = b.groupby(['id_x','id_y','week_of_year_x','year_of_year_x'])[['Return_x','Return_y']].corr().reset_index().iloc[:,0:-1]
        ff = ff[ff['level_4'] != 'Return_x'].drop(columns = ['level_4']).rename(columns = {'Return_x':'ρ'}).sort_values(by = ['year_of_year_x','week_of_year_x'])
        ff = ff.merge(c,on = ['year_of_year_x','week_of_year_x'])
        
        d = f.merge(ff,on=['year_of_year_x','week_of_year_x','id_x','id_y','jalaliDate_x','date'],how = 'outer')
        d['FCAPf'] = (d['FCAPf'] - d['FCAPf'].mean()).divide(d['FCAPf'].std())
        d = d.rename(columns = {'FCAPf':'FCAPF'})
        d['SGroup'] = 0
        if a[['group_name_x','group_name_y']].iloc[0][0] == a[['group_name_x','group_name_y']].iloc[0][1]:
            d['SGroup'] = 1
        d['PCorr_1'] = d['ρ'].shift(1)
        d['PCorr_3'] = d['ρ'].shift(3)
        d['PCorr_6'] = d['ρ'].shift(6)
        d['PCorr_12'] = d['ρ'].shift(12)
        d['FCorr_1'] = d['ρ'].shift(-1)
        d['FCorr_3'] = d['ρ'].shift(-3)
        d['FCorr_6'] = d['ρ'].shift(-6)
        d['FCorr_12'] = d['ρ'].shift(-12)
        d = d.loc[~d['FCAPF'].isnull()]

In [ ]:
d

In [ ]:
df.columns

In [ ]:
MarketCap = df
MarketCap['MarketCap'] = MarketCap['close_price'] * MarketCap['shrout']
MarketCap = MarketCap.groupby(['date','symbol']).first().reset_index()[['date', 'jalaliDate', 'symbol','MarketCap']].groupby(['date','jalaliDate'])['MarketCap'].sum().to_frame().reset_index()


In [ ]:
path2 = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data\Analyzed Data\FCAP"
n2 = path2 + '\\NormalzedFCAP.csv'
# Nd.to_csv(n2,index = False)

In [ ]:
d = pd.read_csv(n2)

In [ ]:
d['year_of_year_x'] = d['year_of_year_x'].astype(str)
d['week_of_year_x'] = d['week_of_year_x'].astype(str)
d['Year_Week'] = d['year_of_year_x'] + d['week_of_year_x']
d

In [ ]:
days = list(set(d.Year_Week))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t'] = d['Year_Week'].map(mapingdict)

In [ ]:
d['id_x'] = d['id_x'].astype(str)
d['id_y'] = d['id_y'].astype(str)
d['id'] = d['id_x'] + d['id_y']
ids = list(set(d.id))
ids.sort()
id = list(range(len(ids)))
mapingdict = dict(zip(ids, id))
d['id'] = d['id'].map(mapingdict)

In [ ]:
d.head()

In [ ]:
d['size1'] = 0
d.loc[d.Size_x>d.Size_y,'size1'] = d[d.Size_x>d.Size_y]['Size_x']
d.loc[d.Size_x<d.Size_y,'size1'] = d[d.Size_x<d.Size_y]['Size_y']
d['size2'] = 0
d.loc[d.Size_x>d.Size_y,'size2'] = d[d.Size_x>d.Size_y]['Size_y']
d.loc[d.Size_x<d.Size_y,'size2'] = d[d.Size_x<d.Size_y]['Size_x']


In [ ]:
d['ρ_f'] = d.groupby(['id'])['ρ'].shift(-1)
d['ρ_f2'] = d.groupby(['id'])['ρ'].shift(-2)
d['ρ_f3'] = d.groupby(['id'])['ρ'].shift(-3)
d['ρ_f5'] = d.groupby(['id'])['ρ'].shift(-5)

In [ ]:
d['SameSize'] = d['size1'] - d['size2']
d['SameSize'] = (d['SameSize']-d['SameSize'].mean()).divide(d['SameSize'].std())
d

In [ ]:
n3 = path2 + '\\NormalzedFCAP2.csv'
d.to_csv(n3,index = False)